## Notebook Contains

A Rag Agent using `Cewai` with short term memory

## Importing Libraries

In [ ]:
# Importing Necessaet Libraries
import os
import sys
import io
from crewai import LLM, Agent, Task, Crew, Process

from crewai.tools import BaseTool

In [2]:
# --- Configuration --- #
MODEL_NAME = "ollama/openhermes:latest"

In [3]:
# --- Configuring the LLM --- #
llm = LLM(
    model= MODEL_NAME,
    temperature=0.7
)

## Creating Supervisor Agent with memory
This class will act as long term memory (persisted between runs).

In [4]:
import sqlite3
import datetime

class SQLiteMemory:
    def __init__(self, db_path="memory.db"):
        self.conn = sqlite3.connect(db_path)
        self.cur = self.conn.cursor()
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS memory (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                role TEXT,
                content TEXT,
                timestamp TEXT
            )
        """)
        self.conn.commit()

    def save(self, role, content):
        ts = datetime.datetime.now().isoformat()
        self.cur.execute("INSERT INTO memory (role, content, timestamp) VALUES (?, ?, ?)", 
                         (role, content, ts))
        self.conn.commit()

    def load_recent(self, limit=10):
        self.cur.execute("SELECT role, content FROM memory ORDER BY id DESC LIMIT ?", (limit,))
        return self.cur.fetchall()


## Creating Python Coad Runner (Coading Agent)

In [5]:
class CodingAgent(BaseTool):
    name: str = "CodingAgent"
    description: str = """A coding agent that can write and execute Python code based on user queries.
    It returns the output of the code if successful, or an error message if an error occurs.
    The query must be a valid Python code block.
    """

    def _run(self, query: str) -> str:
        """Run the coding agent to write and execute Python code based on the query."""
        
        # We need to capture the output, so we redirect stdout to a string buffer.
        old_stdout = sys.stdout
        redirected_output = io.StringIO()
        sys.stdout = redirected_output
        
        try:
            # Execute the user's query as a code block.
            # The 'exec' function can run multi-line statements, including imports and function definitions.
            exec(query)
            
            # Get the output from the redirected buffer.
            output = redirected_output.getvalue()
            
            # Restore the original stdout.
            sys.stdout = old_stdout
            
            # If there's no output, we return a message indicating success.
            if not output.strip():
                return "Code executed successfully with no output."
            
            # Return the captured output.
            return output
            
        except Exception as e:
            # If any error occurs during execution, capture it and return the error message.
            # Restore the original stdout first.
            sys.stdout = old_stdout
            return f"An error occurred while executing the code: {e}"

In [6]:
coding_tool = CodingAgent()

print(coding_tool._run("print('Hello')"))
# print(run_python_code("print('Hello World!!!')"))

Hello



In [7]:
class DirectAnswerTool(BaseTool):
    name: str = "DirectAnswerTool"
    description: str = "A tool for the supervisor to give a direct, final answer to a query."

    def _run(self, query: str) -> str:
        # This tool's purpose is to signal to the crew that a direct answer has been given.
        return f"Supervisor directly answered the query: {query}"

In [8]:
# Instantiate the tools
coding_tool = CodingAgent()
helper_tool = DirectAnswerTool()

# --- Defining All Agents --- #
# Define Coding Agent
coding_agent = Agent(
    role="coding_agent",
    goal="""Write python code based on user queries and answer executing code.""",
    backstory= """You are a helpful assistant. You can write and execute Python code based on user queries and return the output,
      executing the code.""",
    verbose=False,
    tools=[coding_tool],
    llm=llm
)

# Defining Helpful Agent
helper_agent = Agent(
    role= "helper_agent",
    goal= "Answer user query based on your knowladge",
    backstory= "You are a helpful assistant. You try to answer user Question.",
    verbose= False,
    tools= [helper_tool],
    llm= llm
)

supervisor_agent = Agent(
    role="Supervisor Agent",
    goal="""Analyze user queries and decide whether to route the request to the Coding Agent or the Helper Agent. Be very strict in your decision-making.""",
    backstory="""You are a top-tier assistant that acts as a router for a team of specialized agents.
    Your primary function is to classify user requests. You MUST ALWAYS route the query to one of the following agents, by name:
    - Coding Agent: For tasks that explicitly require writing, running, or debugging code (e.g., "write a Python script", "calculate X using code", "debug this function").
    - Helper Agent: For all other general knowledge questions that can be answered directly from memory (e.g., "What is the capital of France?", "Who invented the lightbulb?").
    You CAN only select one of these agents.""",
    verbose=False,
    llm=llm
)

# --- Defining Tasks --- #
# Defining Coading Task
coding_task = Task(
    description=f"""Write clean python code based on the user query.
        User Query: {{query}}""",
    expected_output="return the code like ```CODE:\n``` and the output of the code like ```OUTPUT:```.",
    agent=coding_agent
)

# Defining Helper task
helper_task = Task(
    description= f''' You Answer the user query. Your Answer should be to the point and short.
        User Query: {{query}} ''',
    expected_output="Return Answer in Natural Language",
    agent=helper_agent
)


supervisor_task = Task(
    description=f"""Analyze the user's query and decide on the best course of action.
    - If the query asks for code, delegate to coding_task.
    - If the query is a general knowledge question, delegate to helper_task.
    User Query: {{query}}""",
    expected_output="The Answer can only be 'coding_task' or 'helper_task'. Nothing else.",
    agent=supervisor_agent,
    possible_tasks=[coding_task, helper_task]  # 👈 This is the missing link
)

# --- Defining Crew --- #
# Define Crew
coding_crew = Crew(
    agents=[coding_agent],
    tasks=[coding_task],
    verbose=False,
    process=Process.sequential, # The supervisor task will run first, and its output will feed into the next task.
)

helper_crew = Crew(
    agents= [helper_agent],
    tasks= [helper_task],
    verbose= False,
    process= Process.sequential
)

supervisor_crew = Crew(
    agents= [supervisor_agent],
    tasks= [supervisor_task],
    verbose= False,
    process= Process.sequential
)

In [9]:
# Checking if Crew is ready
# print(coading_crew.kickoff(inputs={"query": "calculate the sum of all natural numbers greater than 5 and less than 10."}) )
# print(helper_crew.kickoff(inputs={'query': "What is the capital of India?"}))

In [12]:
# Defining Control Flow.
def main():
    """Main function to kickoff the Crew."""
    print("\n\t--- Starting Crew for RAG Agent ---")

    while True:
        user_query = input("Enter your query (or type 'exit' to quit): ")
        print(f"\n🙎‍♂️: {user_query}")

        # Breaking the loop if exit or quit was input
        if user_query.lower() in ['exit', 'quit']:
            break

        decision = supervisor_crew.kickoff(inputs={'query': user_query})
        print(f"\t I need to check with {decision}")

        if decision.raw == 'helper_task':
            result = helper_crew.kickoff(inputs= {'query': user_query})
            print(f"🤖: {result}")
        elif decision.raw == 'coding_task':
            result = coding_crew.kickoff(inputs= {'query': user_query})
            print(f"🤖: {result}")
        else:
            print(f"🤖: I did not find any tool to answer.")

main()


	--- Starting Crew for RAG Agent ---

🙎‍♂️: Write a python code to print pascle triangle. User will give input as number of rows. Show output for 5 rows
	 I need to check with coding_task
🤖: CODE:
```python
def print_pascal(rows):
    for i in range(1, rows + 1):
        line = [1] * i
        print(''.join([str(line[j]) for j in range(i+1)]))
```
OUTPUT:
```python
1
1 1
1 2 1
1 3 3 1
1 4 6 4 1
```

🙎‍♂️: Name the people landed on moon with year
	 I need to check with helper_task
🤖: The people who landed on the Moon are Neil Armstrong, Buzz Aldrin, and Michael Collins. They landed in 1969 as part of the Apollo 11 mission.

🙎‍♂️: How many people landed on moon till now?
	 I need to check with helper_task
🤖: As of now, 12 people have landed on the moon.

🙎‍♂️: What are the name of those 12 people?
	 I need to check with helper_task
🤖: The names of the 12 people can vary depending on context and situation. Without further information, it is impossible to provide a definitive list. Please